In [1]:
#jupyter無法用@@

import multiprocessing as mp
#import threading as td

def job(a,b):
    print('aaa')

if __name__=='__main__': #要加這個
    #t1 = td.Thread(target=job, args=(a,b))  #加thread
    p1 = mp.Process(target=job, args=(1,2)) #加Process,用法跟thread一樣
    p1.start()
    p1.join()


In [ ]:
import multiprocessing as mp

def job(q):
    res = 0
    for i in range(1000):
        res += i+i**2+i**3
    q.put(res) # queue

if __name__ == '__main__':
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,)) #args(q,) 逗點一定要
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print(res1+res2)

In [ ]:
#一般與多進程與多核心運算比較
import multiprocessing as mp
import threading as td
import time

def job(q):
    res = 0
    for i in range(1000000):
        res += i+i**2+i**3
    q.put(res) # queue

def multicore():
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,))
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print('multicore:' , res1+res2)

def normal():
    res = 0
    for _ in range(2):
        for i in range(1000000):
            res += i+i**2+i**3
    print('normal:', res)

def multithread():
    q = mp.Queue()
    t1 = td.Thread(target=job, args=(q,))
    t2 = td.Thread(target=job, args=(q,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    res1 = q.get()
    res2 = q.get()
    print('multithread:', res1+res2)

if __name__ == '__main__':
    st = time.time()
    normal()
    st1= time.time()
    print('normal time:', st1 - st)
    multithread()
    st2 = time.time()
    print('multithread time:', st2 - st1)
    multicore()
    print('multicore time:', time.time()-st2)

normal: 499999666667166666000000
normal time: 1.9009251594543457
multithread: 499999666667166666000000
multithread time: 2.130061388015747


In [ ]:
import multiprocessing as mp

def job(x):
    return x*x

def multicore():
    pool = mp.Pool(processes=2) #上面job()可以有return
    res = pool.map(job, range(10))
    print(res)
    res = pool.apply_async(job, (2,))
    print(res.get())
    multi_res =[pool.apply_async(job, (i,)) for i in range(10)] #迭代器使用
    print([res.get() for res in multi_res])

if __name__ == '__main__':
    multicore()

In [1]:
#全局變量無法傳給各個Core,故無法共享戶傳
#所以要共享記憶體才可以給不同core用
#用法 
#import multiprocessing as mp
#value = mp.Value('d', 1) #'d'為資料型態
#array = mp.Arrar('i', [1,2,3]) #'i'為資料型態, 但array只能一維

In [ ]:
import multiprocessing as mp
import time

def job(v, num, l):
    l.acquire()
    for _ in range(10):
        time.sleep(0.1)
        v.value += num
        print(v.value)
    l.release()

def multicore():
    l = mp.Lock()
    v = mp.Value('i', 0)
    p1 = mp.Process(target=job, args=(v, 1, l))
    p2 = mp.Process(target=job, args=(v, 3, l))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

if __name__ == '__main__':
    multicore()
